In [1]:
from cergen import gergen, rastgele_dogal, rastgele_gercek, Operation, map_nested_list
from typing import Literal, Optional, Union

import math
import pandas as pd

In [2]:
"""
TYPE ALIASES & CONSTANTS
"""

ActivationType = Literal['softmax', 'relu']

In [3]:
def ReLU(x: Union[float, int]) -> Union[float, int]:
    return max(0, x)

def softmax(x: Union[float, int], all_values: list[Union[float, int]]) -> float:
    return math.exp(x) / sum(math.exp(xi) for xi in all_values)

In [4]:
class ForwardPass(Operation):
    def ileri(self, x: gergen, W: gergen, b: gergen) -> gergen:
        """
        The forward pass of a layer, which computes the output of the layer given the input x, weights W, and biases b.
        """
        return W.ic_carpim(x) + b


class Katman:
    _x: gergen = None

    _W: gergen = None

    _b: gergen = None

    _outputs: gergen = None

    _activation: str = None
    
    _forward_pass = ForwardPass()

    def __init__(self, input_size: int, output_size: int, activation: Optional[ActivationType] = None):
        """
        With the given input and output sizes, the layer’s weighs and biases are initialised with appropriate shapes using the rastgele_gercek() function.
        """
        
        self._x = rastgele_gercek((input_size, 1))
        self._W = rastgele_gercek((output_size, input_size))
        self._b = rastgele_gercek((output_size, 1))
        self._activation = activation

        self._outputs = self.calculate_forward()

    def calculate_forward(self):
        """
        The forward method computes and returns the output of the layer given the input x, weights W, and biases b.
        """
        raw_forward_pass: gergen = self._forward_pass(self._x, self._W, self._b)

        if self._activation == 'relu':
            return raw_forward_pass.map(ReLU)

        elif self._activation == 'softmax':
            list_raw_forward_pass = raw_forward_pass.listeye()
            flattened = raw_forward_pass.duzlestir()

            mapped = map_nested_list(list_raw_forward_pass, lambda x: softmax(x, flattened))

            return gergen(mapped)

        else:
            return raw_forward_pass
        
    def get_output(self):
        """
        The get_output method returns the output of the layer.
        """
        return self._outputs

    def set_input(self, x: gergen):
        """
        The set_input method sets the input of the layer to the given input x.
        """
        self._x = x
        self._outputs = self.calculate_forward()

    def set_weights(self, W: gergen):
        """
        The set_weights method sets the weights and biases of the layer to the given weights W.
        """
        self._W = W
        self._outputs = self.calculate_forward()

    def set_bias(self, b: gergen):
        """
        The set_bias method sets the biases of the layer to the given biases b.
        """
        self._b = b
        self._outputs = self.calculate_forward()

    def set_activation(self, activation: ActivationType):
        """
        The set_activation method sets the activation function of the layer to the given activation.
        """
        self._activation = activation
        self._outputs = self.calculate_forward()

    def get_weights(self):
        """
        The get_weights method returns the weights of the layer.
        """
        return self._W
    
    def get_bias(self):
        """
        The get_bias method returns the biases of the layer.
        """
        return self._b
    
    def get_activation(self):
        """
        The get_activation method returns the activation function of the layer.
        """
        return self._activation
    
    def get_input(self):
        """
        The get_input method returns the input of the layer.
        """
        return self._x


In [11]:
class MLP:
    _input_size: int = None

    _hidden_size: int = None

    _output_size: int = None

    _predicted_y_values: gergen = None

    _loss: float = None

    def __init__(self, input_size: int, hidden_size: int, output_size: int):
        """
        The MLP is initialized with three parameters: input size, hidden size, and output size, which correspond to the dimensions of the input layer, hidden
        layer, and output layer, respectively.
        Within the init method of the class, two layers are instantiated using the Layer class:
         - self.hidden layer: This is the hidden layer of the network that takes inputs from the input layer and applies a ReLU (Rectified Linear Unit)
         activation function to each neuron’s output.
         - self.output layer: This is the output layer of the network that takes inputs from the hid- den layer. The activation function used here is softmax,
         which is appropriate for multi-class classification tasks. Softmax converts the outputs (raw scores) to a probability distribution over the output
         classes. In cases where a MLP is used for regression or binary classification, the activation function may be omitted or replaced with a sigmoid
         function, respectively.
        """

        self._input_size = input_size
        self._hidden_size = hidden_size
        self._output_size = output_size

        self.hidden_layer = Katman(input_size, hidden_size, 'relu')
        self.output_layer = Katman(hidden_size, output_size, 'softmax')

    def ileri(self, inputs: gergen):
        """
        The forward method computes the output of the network given the input x.
        """

        """
        The part below "normalises" the inputs. The normalisation in question is NOT an actual normalisation, but rather a transformation of the inputs.
        The inputs are transformed to a 2D array, where each column corresponds to a single input sample. This is done to ensure that the matrix
        multiplication in the forward pass is done correctly.
        """

        normalised_inputs = inputs.boyutlandir((self._input_size, 1))

        self.hidden_layer.set_input(normalised_inputs)
        self.output_layer.set_input(self.hidden_layer.get_output())

        self._predicted_y_values = self.output_layer.get_output()

        return self._predicted_y_values
    
    def calculate_loss(self, true_y_values: gergen):
        """
        The calculate_loss method computes the loss of the network given the true y values, using cross-entropy loss.
        """

        true_log_y_values = true_y_values.log()

        return (true_log_y_values * self._predicted_y_values) / (- self._output_size)